# Run redis with docker

In [1]:
from ansible_please.ansible_runner import AnsibleRunner
from ansible_please.playbook import Playbook
from ansible_please.task_templates import Pip, DockerContainer
from omegaconf import OmegaConf

### Install docker via pip

In [2]:
pip_task = Pip(packages=["docker"])

In [3]:
pip_task

### Define docker task

In [4]:
docker_container_task = DockerContainer(
    task_description="start-test-redis",
    name="test-redis",
    image="redis:latest",
)

In [5]:
print(docker_container_task)

- name: '[up] start-test-redis'
  community.docker.docker_container:
    name: test-redis
    image: redis:latest
    user: nobody
    keep_volumes: false
    detach: true
    tty: false
    interactive: false
    network_mode: host
    container_default_behavior: compatibility
  tags:
  - up



### Define Playbook

In [6]:
playbook = Playbook(
    name="test-redis-playbook",
    hosts="master_host",
    tasks=[
        pip_task.up(),
        docker_container_task.up(),
        docker_container_task.down(),
    ],
)

### Run playbook

In [8]:
runner = AnsibleRunner(input_path='test_input.yml', playbooks=[playbook])
runner.up()

No config file found; using defaults

PLAY [test-redis-playbook] *****************************************************

TASK [Gathering Facts] *********************************************************
ok: [192.168.1.248]

TASK [[up] Install package via pip] ********************************************
changed: [192.168.1.248] => {"changed": true, "cmd": ["/usr/bin/python3", "-m", "pip", "install", "docker"], "delta": "0:00:00.896875", "end": "2021-05-23 17:02:46.414234", "rc": 0, "start": "2021-05-23 17:02:45.517359", "stderr": "", "stderr_lines": [], "stdout": "Requirement already satisfied: docker in /home/shyam/.local/lib/python3.8/site-packages (4.4.1)\nRequirement already satisfied: websocket-client>=0.32.0 in /home/shyam/.local/lib/python3.8/site-packages (from docker) (0.57.0)\nRequirement already satisfied: six>=1.4.0 in /usr/lib/python3/dist-packages (from docker) (1.14.0)\nRequirement already satisfied: requests!=2.18.0,>=2.14.2 in /usr/lib/python3/dist-packages (from docker)

2021-05-23 17:02:49.175 | INFO     | ansible_please.ansible_runner:run:64 - successful: 0
2021-05-23 17:02:49.177 | INFO     | ansible_please.ansible_runner:run:65 - Final status:
2021-05-23 17:02:49.186 | INFO     | ansible_please.ansible_runner:run:66 - {'skipped': {}, 'ok': {'192.168.1.248': 3}, 'dark': {}, 'failures': {}, 'processed': {'192.168.1.248': 1}, 'changed': {'192.168.1.248': 2}}


### you should see that the container "test-redis" is up

### Now tear it down

In [9]:
# tear down
runner.down()

No config file found; using defaults

PLAY [test-redis-playbook] *****************************************************

TASK [Gathering Facts] *********************************************************
ok: [192.168.1.248]

TASK [[down] start-test-redis] *************************************************
ok: [192.168.1.248] => {"changed": false}

PLAY RECAP *********************************************************************
192.168.1.248              : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



2021-05-23 17:04:37.308 | INFO     | ansible_please.ansible_runner:run:64 - successful: 0
2021-05-23 17:04:37.310 | INFO     | ansible_please.ansible_runner:run:65 - Final status:
2021-05-23 17:04:37.329 | INFO     | ansible_please.ansible_runner:run:66 - {'skipped': {}, 'ok': {'192.168.1.248': 2}, 'dark': {}, 'failures': {}, 'processed': {'192.168.1.248': 1}, 'changed': {}}


## DockerCluster

In [ ]:
from ansible_please.clusters.docker_cluster import DockerCluster

In [ ]:
c = DockerCluster(runner)

In [ ]:
c.up()

### get cluster from info redis container

In [ ]:
c = DockerCluster()

In [ ]:
print(c.ansible_runner.playbooks[0])

In [ ]:
import docker

client = docker.DockerClient()

In [ ]:
container = client.containers.get('ansible_please-cluster-info')
container.attrs['Id']

In [ ]:
container.attrs["State"]

In [ ]:
c.down()